<a href="https://colab.research.google.com/github/hoky1227/Transformer_based-recommendation/blob/main/Transformer_with_ECF_Algorithm_1230_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/'

### Module 가져오기

In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import math
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

import warnings 

warnings.filterwarnings('ignore')

### Data Road

In [ ]:
TFoutputs = pd.read_csv(path + "Transformer_test_outputs_10.csv")
TFoutputs.pop('Unnamed: 0')
TFoutputs

In [ ]:
checker = pd.Series([i in list(TFoutputs['split_label_token']) for i in TFoutputs['predict_item']])
# filter = checker.replace(False, np.NaN)
print(checker.value_counts())
filter = checker[checker == False].index
len(filter)

In [ ]:
TFoutputs['filter'] = checker
# falses = TFoutputs.isna().sum()
# TFoutputs.dropna(inplace=True)
# TFoutputs.reset_index(drop=True, inplace=True)
# TFoutputs

In [ ]:
TFoutputs

In [ ]:
pred = TFoutputs['predict_item']
pred = pred.str.replace('<esp>', '')
pred = pred.str.strip()
pred.head()

In [ ]:
# falses = falses['filter']
# falses

falses = pd.Series([TFoutputs['predict_item'][i] for i in filter])
falses = falses.str.replace('<esp>', '')
falses = falses.str.strip()
print(len(falses))
falses.head()

In [ ]:
filter[:5]

In [ ]:
df = pd.read_csv(path + "data_customer_based.csv")
df.pop('Unnamed: 0')
df

In [ ]:
user2item = df.copy()

In [ ]:
user2item.columns = ['유저 식별키', '유저 구매 상품 식별키 묶음', '라벨']
user2item

# Product2Vector 생성

### Jaccard 유사도

In [ ]:
# detail = pd.read_csv(path + "Detail.csv")
# detail.pop('Unnamed: 0')
# print(len(detail))
# detail = detail['0'].to_list()
# len(detail)

In [ ]:
# pd.Series([i in detail for i in falses]).value_counts()

In [ ]:
def JaccardSimilarity(inp1, inp2):
    list_inp1 = inp1.split()
    list_inp2 = inp2.split()
    mom = set(list_inp1).union(set(list_inp2))
    son = set(list_inp1).intersection(set(list_inp2))
    # print(mom)
    # print(son)
    return len(son)/len(mom)

In [ ]:
detail = TFoutputs['label'].unique().tolist()
print(detail[-1])
len(detail)

In [ ]:
pred[75]

In [ ]:
asdf = []

for v in falses:
    max = 0
    for d in detail:
        score = JaccardSimilarity(v, d)
        if max < score:
            max = score
            temp = d
    if max == 0: asdf.append(v)
    id = pred[pred == v].index
    for i in id:
        pred[i] = temp

In [ ]:
len(asdf)

In [ ]:
falses[0]

In [ ]:
pred[75]

In [ ]:
pd.Series([i in detail for i in pred]).value_counts()

In [ ]:
changes = pd.DataFrame(columns=['Original', 'Changed'])
changes['Original'] = falses
changes['Changed'] = pred[filter].to_list()
changes

In [ ]:
changes.to_csv(path + 'changed_products.csv', mode='w')

### Word2vec Model 학습

In [ ]:
model = Word2Vec([item_list.split('&&') for item_list in user2item['유저 구매 상품 식별키 묶음']], size=100, window=5, min_count=1, negative=15, iter=20, sample=0.00001, sg=1)

In [ ]:
# 유저 성향 벡터 구하기 
user_propensity_vector = []
for item_list in tqdm_notebook(user2item['유저 구매 상품 식별키 묶음']):
    item_list = item_list.split('&&')
    bbb = np.array(np.ones(100))
    for aaa in item_list:
        bbb *= model[aaa]
        bbb = bbb/norm(bbb)
    user_propensity_vector.append(bbb)

In [ ]:
# 유사 product 찾기
# def find_item(user, n=10):
#     item_code = user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('&&')[-1]
#     return list(dict(model.wv.most_similar(item_code, topn=n)))

def find_item(item, n=10):
    return list(dict(model.wv.most_similar(item, topn=n)))

# 유저 유사도 계산 및 유사 유저 찾기
def user_sim(A, B): 
    A_vec = user_propensity_vector[user2item[user2item['유저 식별키']==A].index[0]]
    B_vec = user_propensity_vector[user2item[user2item['유저 식별키']==B].index[0]]
    return euclidean(A_vec.reshape(-1, 1), B_vec.reshape(-1, 1))

def find_user(user):
    user_index = user2item[user2item['유저 식별키']==user].index[0]
    min_user_similar = min(user_matrix_f[user_index].drop(user_index))
    return user2item.iloc[{j:i for i, j in enumerate(user_matrix_f[user_index])}[min_user_similar]]['유저 식별키']

In [ ]:
# # 유저 유사도 매트릭스
# user_matrix_f = pd.DataFrame()
# for i in tqdm_notebook(range(len(user_propensity_vector))):
#     tmp = []
#     for j in range(len(user_propensity_vector)):
#         tmp.append(euclidean(user_propensity_vector[i], user_propensity_vector[j]))
#     user_matrix_f = pd.concat([user_matrix_f, pd.DataFrame(tmp)], axis = 1)
# user_matrix_f.columns = user_matrix_f.index
# user_matrix_f

In [ ]:
find_item(pred[0])

## 구매전 : Top-10 시나리오

In [ ]:
def recommend(n=10, acc=False, pred=pred, TFoutputs=TFoutputs):
    item_list = []
    for v in pred.to_list():
        pp = find_item(v, n)
        item_list.append(pp)
    TFoutputs['추천 list'] = item_list
    # user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]
    TFoutputs['정답 유무'] = [True if label in item else False for label, item in tqdm_notebook(zip(TFoutputs['label'], TFoutputs['추천 list']), total=(len(TFoutputs)))]
    # 추천 알고리듬 구매전 Top-10 성능평가
    
    if acc:
        return TFoutputs[TFoutputs['정답 유무']==True].shape[0]/(TFoutputs.shape[0])

    return TFoutputs

In [ ]:
recommend(n=5, acc=True)

In [ ]:
recommend(n=10, acc=True)

In [ ]:
recommend(n=20, acc=True)

In [ ]:
recommend()

In [ ]:
# item_list = []
# for v in pred.to_list():
#     pp = find_item(v)
#     item_list.append(pp)

# item_list[:2]

In [ ]:
# def recommendation(user):
#     n = 10
#     while True:
#         recom_item = set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('&&')) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('&&'))
#         n += 10
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]           #Top10
#         if n >= 1000:
#             return list(recom_item)

In [ ]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(user2item['유저 식별키'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':user2item['유저 식별키'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','&&') 
#                                    for i in rec_list]})
# user2rec

In [ ]:
# TFoutputs['추천 list'] = item_list

In [ ]:
# # user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]
# TFoutputs['정답 유무'] = [True if label in item else False for label, item in tqdm_notebook(zip(TFoutputs['label'], TFoutputs['추천 list']), total=(len(TFoutputs)))]

In [ ]:
# TFoutputs

In [ ]:
# # 추천 알고리듬 구매전 Top-10 성능평가

# TFoutputs[TFoutputs['정답 유무']==True].shape[0]/(TFoutputs.shape[0] + falses)

# 구매후(교-차)  : Top-10 시나리오

In [ ]:
# def recommendation(user):
#     n = 10
#     while True:
#         recom_item = set(set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('@')) & set(find_item(user, n))) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('@'))
#         n += 10   
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]            #Top10
#         if n >= 1000:
#             return list(recom_item)

In [ ]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(df_f2['고객번호'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':df_f2['고객번호'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','@') 
#                                     for i in rec_list]})
# user2rec

In [ ]:
# user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]

In [ ]:
# # 추천 알고리듬 구매후(교-차) Top-10 성능평가


# user2item[user2item['정답 유무']==True].shape[0]/user2item.shape[0]*100 #Top10

# 구매후(합-차) : Top-10 시나리오

In [ ]:
# def recommendation(user):
#     n = 20
#     while True:
#         recom_item = set(set(user2item[user2item['유저 식별키']==find_user(user)]['유저 구매 상품 식별키 묶음'].values[0].split('@')) | set(find_item(user, n))) - set(user2item[user2item['유저 식별키']==user]['유저 구매 상품 식별키 묶음'].values[0].split('@'))
#         n += 10   
#         if len(recom_item) >= 10:
#             return list(recom_item)[:10]              #Top10
#         if n >= 1000:
#             return list(recom_item)

In [ ]:
# # 추천 List 생성

# rec_list = []
# for i in tqdm_notebook(df_f2['고객번호'].unique()):
#     r_list = list(recommendation(i))
#     rec_list.append(r_list)
# rec_list

# user2rec = pd.DataFrame({'유저 식별키':df_f2['고객번호'].unique(),
#                          '추천 list':[str(i).replace('[','').replace(']','').replace("'",'').replace(',','@') 
#                                     for i in rec_list]})
# user2rec

In [ ]:
# user2item['정답 유무'] = [True if label in recommendation(user) else False for user, label in tqdm_notebook(zip(user2item['유저 식별키'], user2item['라벨']), total=len(user2item['유저 식별키']))]

In [ ]:
# # 추천 알고리듬 구매후(합-차) Top-10 성능평가

# user2item[user2item['정답 유무']==True].shape[0]/user2item.shape[0]*100 #Top10